# ReAct: Reasoning + Acting

In [ ]:
import re
from agentic_patterns.core.agents import get_agent, run_agent

## The ReAct Prompting Format

ReAct is a prompting pattern where the model explicitly writes Thought/Action/Observation traces in its text output. The system parses the Action, executes it, and appends the Observation back to the context. This loop continues until a Final answer is produced.

This was a precursor to modern tool-calling APIs.

## Define the Environment

The "environment" provides actions the model can take. Here we simulate an order tracking system with data the model cannot know from training.

In [ ]:
# Simulated order database - data the model cannot know
ORDERS = {
    "ORD-7842": {"customer": "alice", "status": "shipped", "carrier": "FedEx", "tracking": "FX-9981234", "items": ["Wireless Mouse", "USB Hub"]},
    "ORD-7843": {"customer": "bob", "status": "processing", "carrier": None, "tracking": None, "items": ["Mechanical Keyboard"]},
    "ORD-7844": {"customer": "alice", "status": "delivered", "carrier": "UPS", "tracking": "1Z999AA10123", "items": ["Monitor Stand"]},
}

CUSTOMERS = {
    "alice": {"orders": ["ORD-7842", "ORD-7844"], "email": "alice@example.com"},
    "bob": {"orders": ["ORD-7843"], "email": "bob@example.com"},
}

def execute_action(action_text: str) -> str:
    """Parse and execute an action, returning the observation."""
    action_text = action_text.strip()
    
    # Parse LookupOrder[order_id] format
    if action_text.startswith("LookupOrder[") and action_text.endswith("]"):
        order_id = action_text[12:-1].strip().upper()
        if order_id in ORDERS:
            o = ORDERS[order_id]
            result = f"Order {order_id}: Status={o['status']}, Items={o['items']}"
            if o['tracking']:
                result += f", Carrier={o['carrier']}, Tracking={o['tracking']}"
            return result
        return f"Order '{order_id}' not found."
    
    # Parse LookupCustomer[name] format
    if action_text.startswith("LookupCustomer[") and action_text.endswith("]"):
        name = action_text[15:-1].strip().lower()
        if name in CUSTOMERS:
            c = CUSTOMERS[name]
            return f"Customer {name}: Orders={c['orders']}, Email={c['email']}"
        return f"Customer '{name}' not found."
    
    # Parse Finish[answer] format
    if action_text.startswith("Finish[") and action_text.endswith("]"):
        return None
    
    return f"Unknown action: {action_text}"

## The ReAct Prompt

The prompt teaches the model the Thought/Action/Observation format through a few-shot example.

In [ ]:
REACT_PROMPT = """You are a customer service assistant. Answer questions by interleaving Thought, Action, and Observation steps.

Available actions:
- LookupCustomer[name]: Get customer info including their order IDs
- LookupOrder[order_id]: Get order details including status and tracking
- Finish[answer]: Return the final answer

Always follow this format:
Thought: reasoning about what to do
Action: one of the available actions

Example:
Question: What is the status of order ORD-1234?
Thought: I need to look up order ORD-1234 to find its status.
Action: LookupOrder[ORD-1234]
Observation: Order ORD-1234: Status=shipped, Items=['Book'], Carrier=USPS, Tracking=9400111899
Thought: The order has been shipped via USPS. I can now answer.
Action: Finish[Order ORD-1234 has been shipped via USPS. Tracking number: 9400111899]

Now answer the following question. Generate ONLY the next Thought and Action (stop after Action, do not generate Observation yourself).
"""

## The ReAct Loop

The core of ReAct: generate a thought+action, execute the action, append the observation, repeat.

In [ ]:
async def react_loop(question: str, max_steps: int = 5) -> str:
    """Run the ReAct loop until Finish or max steps reached."""
    agent = get_agent(system_prompt=REACT_PROMPT)
    
    trajectory = f"Question: {question}\n"
    
    for step in range(max_steps):
        agent_run, _ = await run_agent(agent, trajectory)
        response = agent_run.result.output.strip()
        
        print(f"--- Step {step + 1} ---")
        print(response)
        
        trajectory += response + "\n"
        
        action_match = re.search(r"Action:\s*(.+?)(?:\n|$)", response)
        if not action_match:
            print("No action found in response.")
            break
        
        action = action_match.group(1).strip()
        
        if action.startswith("Finish["):
            final_answer = action[7:-1]
            print(f"\n=== Final Answer: {final_answer} ===")
            return final_answer
        
        observation = execute_action(action)
        print(f"Observation: {observation}")
        print()
        
        trajectory += f"Observation: {observation}\n"
    
    return "Max steps reached without final answer."

## Example 1: Single Lookup

A simple question requiring one action.

In [ ]:
await react_loop("What is the status of order ORD-7843?")

## Example 2: Multi-Step Reasoning

A question requiring multiple lookups to gather information.

In [ ]:
await react_loop("Has Alice received all her orders? If not, which ones are still pending?")